## Setup and Imports

In [ ]:
import os
from pathlib import Path

import itk
import numpy as np
import pyvista as pv

# Import from PhysioMotion4D package
from physiomotion4d import (
    ContourTools,
    SegmentChestTotalSegmentator,
    WorkflowRegisterHeartModelToPatient,
)

## Define File Paths

In [ ]:
# Patient CT image (defines coordinate frame)
patient_data_dir = Path.cwd().parent / ".." / "data" / "Slicer-Heart-CT"
patient_ct_path = patient_data_dir / "patient_img.mha"
patient_ct_heart_mask_path = patient_data_dir / "patient_heart_wall_mask.nii.gz"

# Atlas template model (moving)
atlas_data_dir = Path.cwd().parent / ".." / "data" / "KCL-Heart-Model"
atlas_vtu_path = atlas_data_dir / "average_mesh.vtk"
atlas_labelmap_path = atlas_data_dir / "average_labelmap_bkg.mha"

pca_data_dir = Path.cwd().parent / ".." / "data" / "KCL-Heart-Model" / "pca"
pca_json_path = pca_data_dir / "pca.json"
pca_group_key = "All"
pca_n_modes = 10

# Output directory
output_dir = Path.cwd() / "results"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
patient_image = itk.imread(str(patient_ct_path))
itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)

In [ ]:
if False:
    segmentator = SegmentChestTotalSegmentator()
    segmentator.contrast_threshold = 500
    patient_segmentation_data = segmentator.segment(
        patient_image, contrast_enhanced_study=False
    )
    labelmap = patient_segmentation_data["labelmap"]
    lung_mask = patient_segmentation_data["lung"]
    heart_mask = patient_segmentation_data["heart"]
    major_vessels_mask = patient_segmentation_data["major_vessels"]
    bone_mask = patient_segmentation_data["bone"]
    soft_tissue_mask = patient_segmentation_data["soft_tissue"]
    other_mask = patient_segmentation_data["other"]
    contrast_mask = patient_segmentation_data["contrast"]

    itk.imwrite(labelmap, str(output_dir / "patient_labelmap.mha"), compression=True)

    heart_arr = itk.GetArrayFromImage(heart_mask)
    # contrast_arr = itk.GetArrayFromImage(contrast_mask)
    mask_arr = (heart_arr > 0).astype(
        np.uint8
    )  # ((heart_arr + contrast_arr) > 0).astype(np.uint8)
    patient_mask = itk.GetImageFromArray(mask_arr)
    patient_mask.CopyInformation(patient_image)

    itk.imwrite(
        patient_mask, str(output_dir / "patient_heart_mask_draft.mha"), compression=True
    )

    # hand edit fixed_mask to make patient_heart_wall_mask.nii.gz that is saved in patient_data_dir
else:
    patient_mask = itk.imread(str(patient_ct_heart_mask_path))

In [ ]:
flip0 = np.array(patient_mask.GetDirection())[0, 0] < 0
flip1 = np.array(patient_mask.GetDirection())[1, 1] < 0
flip2 = np.array(patient_mask.GetDirection())[2, 2] < 0
if flip0 or flip1 or flip2:
    print("Flipping patient image...")
    print(flip0, flip1, flip2)
    flip_filter = itk.FlipImageFilter.New(Input=patient_image)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_image = flip_filter.GetOutput()
    id_mat = itk.Matrix[itk.D, 3, 3]()
    id_mat.SetIdentity()
    patient_image.SetDirection(id_mat)
    itk.imwrite(patient_image, str(output_dir / "patient_image.mha"), compression=True)
    print("Flipping patient mask image...")
    flip_filter = itk.FlipImageFilter.New(Input=patient_mask)
    flip_filter.SetFlipAxes([int(flip0), int(flip1), int(flip2)])
    flip_filter.SetFlipAboutOrigin(True)
    flip_filter.Update()
    patient_mask = flip_filter.GetOutput()
    patient_mask.SetDirection(id_mat)
    itk.imwrite(patient_mask, str(output_dir / "patient_mask.mha"), compression=True)

In [ ]:
patient_model = ContourTools().extract_contours(patient_mask)
patient_model.save(str(output_dir / "patient_mesh.vtp"))
patient_model = pv.read(str(output_dir / "patient_mesh.vtp"))

template_model = pv.read(str(atlas_vtu_path))
template_model_surface = template_model.extract_surface()
template_model_surface.save(str(output_dir / "model_surface.vtp"))
template_model_surface = pv.read(str(output_dir / "model_surface.vtp"))
template_labelmap = itk.imread(str(atlas_labelmap_path))

In [ ]:
registrar = WorkflowRegisterHeartModelToPatient(
    template_model=template_model,
    template_labelmap=template_labelmap,
    template_labelmap_heart_muscle_ids=[1],
    template_labelmap_chamber_ids=[2, 3, 4, 5],
    template_labelmap_background_ids=[6],
    patient_image=patient_image,
    patient_models=[patient_model],
    pca_json_filename=pca_json_path,
    pca_group_key=pca_group_key,
    pca_number_of_modes=pca_n_modes,
)

registrar.set_mask_dilation_mm(0)
registrar.set_roi_dilation_mm(25)

patient_image = registrar.patient_image
itk.imwrite(
    patient_image, str(output_dir / "patient_image_preprocessed.mha"), compression=True
)

In [ ]:
# Rough alignment using ICP
icp_results = registrar.register_model_to_model_icp()
icp_inverse_point_transform = icp_results["inverse_point_transform"]
icp_forward_point_transform = icp_results["forward_point_transform"]
icp_model_surface = icp_results["registered_template_model_surface"]
icp_labelmap = icp_results["registered_template_labelmap"]

icp_model_surface.save(str(output_dir / "icp_model_surface.vtp"))
itk.imwrite(icp_labelmap, str(output_dir / "icp_labelmap.mha"), compression=True)

In [ ]:
pca_results = registrar.register_model_to_model_pca()
pca_coefficients = pca_results["pca_coefficients"]
pca_model_surface = pca_results["registered_template_model_surface"]
pca_labelmap = pca_results["registered_template_labelmap"]

pca_model_surface.save(str(output_dir / "pca_model_surface.vtp"))
itk.imwrite(pca_labelmap, str(output_dir / "pca_labelmap.mha"), compression=True)

## Mask Alignment

In [ ]:
# Perform deformable registration
print("Starting deformable mask-to-mask registration...")

m2m_results = registrar.register_mask_to_mask(use_icon_refinement=False)
m2m_inverse_transform = m2m_results["inverse_transform"]
m2m_forward_transform = m2m_results["forward_transform"]
m2m_model_surface = m2m_results["registered_template_model_surface"]
m2m_labelmap = m2m_results["registered_template_labelmap"]

print("Registration complete!")

m2m_model_surface.save(str(output_dir / "m2m_model_surface.vtp"))
itk.imwrite(m2m_labelmap, str(output_dir / "m2m_labelmap.mha"), compression=True)

In [ ]:
print("Starting deformable registration...")
print("This may take several minutes depending on GPU availability.")

m2i_results = registrar.register_labelmap_to_image()
m2i_inverse_transform = m2i_results["inverse_transform"]
m2i_forward_transform = m2i_results["forward_transform"]
m2i_surface = m2i_results["registered_template_model_surface"]
m2i_labelmap = m2i_results["registered_template_labelmap"]
print("\nRegistration complete!")

# Save registration results to output folder
m2i_surface.save(str(output_dir / "m2i_model_surface.vtp"))
itk.imwrite(m2i_labelmap, str(output_dir / "m2i_labelmap.mha"), compression=True)

In [ ]:
import time


tmp_p = itk.Point[itk.D, 3]()
point = registrar.template_model.points[0]
tmp_p[0] = float(point[0])
tmp_p[1] = float(point[1])
tmp_p[2] = float(point[2])

start_time = time.time()
# Don't save the results since ICP transform is applied as a post-PCA transform
_ = registrar.icp_registrar.forward_point_transform.TransformPoint(tmp_p)
print(f"--- ICP forward transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
# Don't apply the post PCA transform since this is just for setup
_ = registrar.pca_registrar.transform_point(tmp_p, include_post_pca_transform=False)
print(f"--- PCA setup time: {time.time() - start_time} seconds", flush=True)
start_time = time.time()
# Apply the post PCA transform since this is the actual transform
tmp_p = registrar.pca_registrar.transform_point(tmp_p, include_post_pca_transform=True)
print(f"PCA + ICP transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
tmp_p = registrar.m2m_inverse_transform.TransformPoint(tmp_p)
print(f"M2M inverse transform time: {time.time() - start_time} seconds", flush=True)

start_time = time.time()
tmp_p = registrar.m2i_inverse_transform.TransformPoint(tmp_p)
print(f"M2I inverse transform time: {time.time() - start_time} seconds", flush=True)

In [ ]:
# Verify registration using the transform member function
surface_transformed = registrar.m2i_template_model_surface
surface_transformed.save(str(output_dir / "registered_template_surface.vtp"))

model_transformed = registrar.transform_model()
model_transformed.save(str(output_dir / "registered_template.vtu"))

## Visualize Final Results

In [ ]:
# Load meshes from registrar member variables
patient_surface = registrar.patient_model_surface
registered_surface = registrar.registered_template_model_surface
icp_surface = registrar.icp_template_model_surface
pca_surface = registrar.pca_template_model_surface
m2m_surface = registrar.m2m_template_model_surface
m2i_surface = registrar.m2i_template_model_surface

# Create side-by-side comparison
plotter = pv.Plotter(shape=(1, 2))

# After rough alignment
plotter.subplot(0, 0)
plotter.add_mesh(patient_surface, color="red", opacity=0.5, label="Patient")
plotter.add_mesh(pca_surface, color="green", opacity=1.0, label="After ICP")
plotter.add_title("PCA Alignment")

# After deformable registration
plotter.subplot(0, 1)
plotter.add_mesh(patient_surface, color="red", opacity=0.5, label="Patient")
plotter.add_mesh(m2i_surface, color="blue", opacity=1.0, label="Registered")
plotter.add_title("Final Registration")

plotter.link_views()
plotter.show()

## Visualize Deformation Magnitude

In [ ]:
# The transformed mesh has deformation magnitude stored as point data
if "DeformationMagnitude" in registered_surface.point_data:
    plotter = pv.Plotter()
    plotter.add_mesh(
        registered_surface,
        scalars="DeformationMagnitude",
        cmap="jet",
        show_scalar_bar=True,
        scalar_bar_args={"title": "Deformation (mm)"},
    )
    plotter.add_title("Deformation Magnitude")
    plotter.show()

    # Print statistics
    deformation = registered_surface["DeformationMagnitude"]
    print("Deformation statistics:")
    print(f"  Min: {deformation.min():.2f} mm")
    print(f"  Max: {deformation.max():.2f} mm")
    print(f"  Mean: {deformation.mean():.2f} mm")
    print(f"  Std: {deformation.std():.2f} mm")
else:
    print("DeformationMagnitude not found in mesh point data")